# Label and describe using an LLM

Download and instantiate an LLM from Huggingface.

Load the LDA topic models. 

Prompt the LLM to generate a label and a description for each topic in the models.

In [1]:
import pandas as pd
import pickle
from transformers import pipeline
import nltk
nltk.download('punkt')


/home/atroncos/anaconda3/envs/arxiv_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/atroncos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load the topic models fitted in a previous notebook.

* lda_gw: Gravitational Waves topics
* lda_cscl: Computation and Language topics

In [2]:
with open('../models/lda_gw.pickle', 'rb') as handle:
    lda_gw = pickle.load(handle)

Get a list of all topics in the model, each topic described by MAX_WORDS 

* The result is a list of topics. Each topic is represented by a tuple.
* The first element of the tuple is a topic number (int).
* The second element of the tuple is a list of tuples,
* Each tuple represents the words characterising he topic (string) and its corresponding probability (float)

In [3]:
MAX_WORDS = 30
# list[tuples<int, list[tuple<string, float>]>]
topics_gw = lda_gw.show_topics(num_words=MAX_WORDS, formatted=False)

### Using gated models

1. Go to huggingface, login, go to `settings/access tokens` 
2. Create a new READ token, save it to ../token.txt
3. Go here: https://huggingface.co/mistralai/Mistral-7B-v0.1 and accept the usage conditions

In [4]:
from huggingface_hub import login
with open('../token.txt', 'r') as handle:
    token = handle.read()
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/atroncos/.cache/huggingface/token
Login successful


In [5]:
def get_topic_str(topic):
    """Return the terms describing a topic as a string
    topic: list of tuples<string, float>
    """
    terms = [term[0] for term in topic[1]]
    resp = ', '.join([term[0] for term in topic[1]])
    return(resp)

In [60]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import disk_offload
import torch
from abc import ABC, abstractmethod

class Promptable(ABC):
    @abstractmethod
    def one_shot(self, prompt): pass

class Mistral(Promptable):
    def __init__(self):
        self.model_id="mistralai/Mistral-7B-Instruct-v0.2"
        self.model = AutoModelForCausalLM.from_pretrained(self.model_id, torch_dtype=torch.float16, low_cpu_mem_usage = True).cpu()
        disk_offload(model=self.model, offload_dir="alpha")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
    
    def one_shot(self, prompt):
        model_inputs = self.tokenizer([prompt], return_tensors="pt").to("cpu")
        generated_ids = self.model.generate(**model_inputs, max_new_tokens=10, do_sample=True)
        decoded_outputs = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(f"decoded_outputs: {decoded_outputs}")
        resp = nltk.sent_tokenize(decoded_outputs.strip())[2]  # split into sentences
        print(f"resp: {resp}")
        return(resp)

In [64]:
def predict_topic_labels(topics, model):
    """
    Predict label for a list of topics.
    returns: dataFrame with columns: topic id, label
    """
    topics_range = [topic[0] for topic in topics]
    labels = []
    resp = []
    for topic_id in topics_range:
        print(f"Processing topic {topic_id} / {len(topics_range)}")
        terms = get_topic_str(topics[topic_id])
        prompt = f"What is the label for the physics of gravitational waves topic described by these terms: {terms}? Output only the label."
        label = model.one_shot(prompt)
        labels.append(label)
        print(f"label: {label}")
        break
    print(topics_range)
    print(labels)
    
    # this fails because break in loop
#    return(pd.DataFrame.from_dict({'topic': topics_range, 'label': labels}))

In [65]:
%%time

mistral = Mistral()
predict_topic_labels(topics_gw, mistral)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.82s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing topic 0 / 4
decoded_outputs: What is the label for the the physics of gravitational waves topic described by these terms: detector, signal, data, noise, frequency, search, time, method, detection, pulsar, sensitivity, based, source, ligo, interferometer, parameter, analysis, timing, model, space, new, test, present, sky, result, measurement, limit, laser, lisa, mode? Output only the label. gravitational_waves: detector, signal,
resp: gravitational_waves: detector, signal,
label: gravitational_waves: detector, signal,
[0, 1, 2, 3]
['gravitational_waves: detector, signal,']
CPU times: user 9min 51s, sys: 1min 3s, total: 10min 55s
Wall time: 7min 5s
